In [1]:
import os
import csv
import pandas as pd
import numpy as np
import string

In [2]:
# list of data downloads as of 4/20/2018
#Accidental_Drug_Related_Deaths__2012-2017.xlsx
#Personal_Income_Tax_By_Town.csv
#Drug_Poisoning_Mortality_by_County__United_States.csv
#Drug_Poisoning_Mortality_by_State__United_States.csv
#VSRR_Provisional_Drug_Overdose_Death_Counts.csv
#Uniform_Crime_Reporting_System_Arrests_in__2010.csv
#Uniform_Crime_Reporting_System_Arrests_in_2012.csv
#Uniform_Crime_Reporting_System_Arrests_2013.csv
#Uniform_Crime_Reporting_System_Arrests_2014.csv
#Uniform_Crime_Reporting_System_Arrests_2015.csv
#Uniform_Crime_Reporting_Offenses_2016.csv
#Uniform_Crime_Reporting_System_Offenses_2011.csv
#Uniform_Crime_Reporting_System_Offenses_2013.csv
#Uniform_Crime_Reporting_System_Offenses_By_Department_2013.csv
#Uniform_Crime_Reporting_System_Offenses_By_Department_2014.csv
#Uniform_Crime_Reporting_System_Offenses_By_Department_2015.csv
#Correctional_Facility_Daily_Population_Count_With_Facility_Date_Filter.csv
#Number_of_Law_Enforcement_Personnel_by_Department_2010-2012.csv
#CT Unemployment Rates.xlsx
#USA Unemployment Rates.xlsx
#State_Department_of_Education_Next_Generation_School_Accountability_data_2014-2015.csv
#expanded_homicide_data_table_8_murder_victims_by_weapon_2009-2013.xls

In [3]:
# The data engineering process reads each downloaded files, checks for NaN's, 
# and writes to the clean_csv folder

# The Accidental_Drug_Related_Deaths__2012-2017.xlsx file contained a column at the end of the dataframe
# called DeathLoc. It contained various information but also included the latitude and longitude of the death.
# Since this was a key variable for visualizing our data, this field had to be munged.
# The next section includes the cleanup and split of that data into latitude and longitude fields.

In [4]:
#connect to the csv file:
accidental_drug_xlsx = os.path.join("datadownload", "Accidental_Drug_Related_Deaths__2012-2017.xlsx")

In [5]:
#read csv file
acc_drug_deaths_df = pd.read_excel(accidental_drug_xlsx, dtype=object)

In [6]:
#view dataframe
acc_drug_deaths_df.head(4)

,CaseNumber,Date,Sex,Race,Age,Residence City,Residence State,Residence County,Death City,Death State,...,Benzodiazepine,Methadone,Amphet,Tramad,Morphine (not heroin),Other,Any Opioid,MannerofDeath,AmendedMannerofDeath,DeathLoc
0,14-9876,2014-06-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Y,NaN,NaN,NaN,NaN,NaN,NaN,Accident,NaN,"CT\n(41.544654, -72.651713)"
1,12-16897,2012-11-30,Male,White,45,NaN,NaN,NaN,NEW HAVEN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accident,NaN,"NEW HAVEN, CT\n(41.308252, -72.924161)"
2,13-11849,2013-08-12,Male,White,30,NEW HAVEN,NaN,NaN,NEW HAVEN,NaN,...,NaN,Y,NaN,NaN,NaN,NaN,NaN,Accident,NaN,"NEW HAVEN, CT\n(41.308252, -72.924161)"
3,14-17578,2014-11-23,Male,White,27,NAUGATUCK,NaN,NaN,NEW MILFORD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accident,NaN,"NEW MILFORD, CT\n(41.576633, -73.408713)"


In [7]:
#copy the dataframe to a work df
acc_drugtemp_df = acc_drug_deaths_df.copy()

#split the DeathLoc column into two columns 
acc_drugtemp3_df = acc_drugtemp_df['DeathLoc'].str[2:-1].str.split('\n', expand=True)

#create a column called geo getting rid of the left (
acc_drugtemp3_df['Geo'] = acc_drugtemp3_df[1][1][1:-1]


#Add a new column to the dataframe with the latitude and longitude
acc_drugtemp4_df = acc_drugtemp_df.assign(Coord = acc_drugtemp3_df[1][:-1][0:-1])
acc_drugtemp4_df.columns
acc_drugtemp4_df.head(2)



,CaseNumber,Date,Sex,Race,Age,Residence City,Residence State,Residence County,Death City,Death State,...,Methadone,Amphet,Tramad,Morphine (not heroin),Other,Any Opioid,MannerofDeath,AmendedMannerofDeath,DeathLoc,Coord
0,14-9876,2014-06-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Accident,NaN,"CT\n(41.544654, -72.651713)","(41.544654, -72.651713"
1,12-16897,2012-11-30,Male,White,45,NaN,NaN,NaN,NEW HAVEN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Accident,NaN,"NEW HAVEN, CT\n(41.308252, -72.924161)","(41.308252, -72.924161"


In [8]:
#create a column called geo
#acc_drugtemp3b_df['Lat'] = acc_drugtemp3_df[1][1][1:-1]
acc_drugtemp4_df['Coord'].head(2)

0    (41.544654, -72.651713
1    (41.308252, -72.924161
Name: Coord, dtype: object

In [9]:
test = acc_drugtemp4_df['Coord'][0]
test.split(',')[1]

' -72.651713'

In [10]:
# split the Coord column into two at the ,
acc_drugtemp5_df = acc_drugtemp4_df['Coord'].str[1:].str.split(',', expand=True)

#create a column called geo
acc_drugtemp3_df['Lat'] = acc_drugtemp5_df[0][:-1][0:-1]
acc_drugtemp3_df['Long'] = acc_drugtemp5_df[1][:-1][0:-1]
print(acc_drugtemp3_df['Lat'][0],acc_drugtemp3_df['Long'][0])
#acc_drugtemp6_df = acc_drugtemp_df.assign(Latitude = acc_drugtemp3_df[1][:-1][0:-1])
acc_drugtemp6_df = acc_drugtemp_df.assign(Latitude = acc_drugtemp3_df['Lat'][0:])

acc_drugtemp6_df.head(2)
#acc_drugtemp6_df.head(5)

41.544654  -72.651713


,CaseNumber,Date,Sex,Race,Age,Residence City,Residence State,Residence County,Death City,Death State,...,Methadone,Amphet,Tramad,Morphine (not heroin),Other,Any Opioid,MannerofDeath,AmendedMannerofDeath,DeathLoc,Latitude
0,14-9876,2014-06-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Accident,NaN,"CT\n(41.544654, -72.651713)",41.544654
1,12-16897,2012-11-30,Male,White,45,NaN,NaN,NaN,NEW HAVEN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Accident,NaN,"NEW HAVEN, CT\n(41.308252, -72.924161)",41.308252


In [11]:
acc_drugtemp7_df = acc_drugtemp_df.assign(Longitude = acc_drugtemp3_df['Long'][0:])
acc_drugtemp7_df.head(5)

,CaseNumber,Date,Sex,Race,Age,Residence City,Residence State,Residence County,Death City,Death State,...,Methadone,Amphet,Tramad,Morphine (not heroin),Other,Any Opioid,MannerofDeath,AmendedMannerofDeath,DeathLoc,Longitude
0,14-9876,2014-06-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Accident,NaN,"CT\n(41.544654, -72.651713)",-72.651713
1,12-16897,2012-11-30,Male,White,45,NaN,NaN,NaN,NEW HAVEN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Accident,NaN,"NEW HAVEN, CT\n(41.308252, -72.924161)",-72.924161
2,13-11849,2013-08-12,Male,White,30,NEW HAVEN,NaN,NaN,NEW HAVEN,NaN,...,Y,NaN,NaN,NaN,NaN,NaN,Accident,NaN,"NEW HAVEN, CT\n(41.308252, -72.924161)",-72.924161
3,14-17578,2014-11-23,Male,White,27,NAUGATUCK,NaN,NaN,NEW MILFORD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Accident,NaN,"NEW MILFORD, CT\n(41.576633, -73.408713)",-73.408713
4,12-11497,2012-08-14,Male,White,21,ENFIELD,NaN,NaN,ENFIELD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Accident,NaN,"ENFIELD, CT\n(41.976501, -72.591985)",-72.591985


In [12]:
acc_drug_deaths_df['Latitude'] = acc_drugtemp6_df['Latitude']
acc_drug_deaths_df['Longitude'] = acc_drugtemp7_df['Longitude']
acc_drug_deaths_df.head(3)

,CaseNumber,Date,Sex,Race,Age,Residence City,Residence State,Residence County,Death City,Death State,...,Amphet,Tramad,Morphine (not heroin),Other,Any Opioid,MannerofDeath,AmendedMannerofDeath,DeathLoc,Latitude,Longitude
0,14-9876,2014-06-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Accident,NaN,"CT\n(41.544654, -72.651713)",41.544654,-72.651713
1,12-16897,2012-11-30,Male,White,45,NaN,NaN,NaN,NEW HAVEN,NaN,...,NaN,NaN,NaN,NaN,NaN,Accident,NaN,"NEW HAVEN, CT\n(41.308252, -72.924161)",41.308252,-72.924161
2,13-11849,2013-08-12,Male,White,30,NEW HAVEN,NaN,NaN,NEW HAVEN,NaN,...,NaN,NaN,NaN,NaN,NaN,Accident,NaN,"NEW HAVEN, CT\n(41.308252, -72.924161)",41.308252,-72.924161


In [13]:
#view data scale
acc_drug_deaths_df.shape

(4083, 34)

In [14]:
#display empty cells by column:
acc_drug_deaths_df.isnull().sum()

CaseNumber                  0
Date                        2
Sex                         4
Race                       11
Age                         3
Residence City            135
Residence State          1967
Residence County          751
Death City                  4
Death State              1882
Death County              653
Location                   22
DescriptionofInjury      2588
InjuryPlace                79
ImmediateCauseA             0
Heroin                   1929
Cocaine                  2907
Fentanyl                 2616
Oxycodone                3536
Oxymorphone              3986
EtOH                     3078
Hydrocodone              3977
Benzodiazepine           3005
Methadone                3695
Amphet                   3980
Tramad                   3993
Morphine (not heroin)    4027
Other                    3705
Any Opioid               3369
MannerofDeath               7
AmendedMannerofDeath     4055
DeathLoc                    0
Latitude                    2
Longitude 

In [15]:
#save data in a new csv:
new_file_name_1 = os.path.join("csv_folder","clean_acc_drug_deaths.csv")
acc_drug_deaths_df.to_csv(new_file_name_1, index=False)

In [16]:
#connect to the csv file:
Personal_Inc_csv = os.path.join("datadownload", "Personal_Income_Tax_By_Town.csv")
#read csv file
personal_inc_df = pd.read_csv(Personal_Inc_csv, dtype=object)

In [17]:
#view dataframe
personal_inc_df.head(3)

,Tax Year,Municipality,Number of Returns,CT Income Tax,Tax Per Capita
0,2015,ANDOVER,"1,498","$5,404,333.00","$3,608.00"
1,2015,ANSONIA,"8,619","$15,276,355.00","$1,772.00"
2,2015,ASHFORD,"2,112","$5,497,095.00","$2,603.00"


In [18]:
#view data scale
personal_inc_df.shape

(1014, 5)

In [19]:
#display empty cells by column:
personal_inc_df.isnull().sum()

Tax Year             0
Municipality         0
Number of Returns    0
CT Income Tax        0
Tax Per Capita       0
dtype: int64

In [20]:
#save data in a new csv:
new_file_name_1 = os.path.join("csv_folder","clean_personal_inc.csv")
personal_inc_df.to_csv(new_file_name_1, index=False)

In [21]:
#3

In [22]:
#connect to the csv file:
Drug_Poison_County_csv = os.path.join("datadownload", "Drug_Poisoning_Mortality_by_County__United_States.csv")
#read csv file
Drug_Poison_County_df = pd.read_csv(Drug_Poison_County_csv, dtype=object)

In [23]:
#view dataframe
Drug_Poison_County_df.head(3)

,FIPS,Year,State,FIPS State,County,Population,"Estimated Age-adjusted Death Rate, 16 Categories (in ranges)"
0,1001,1999,Alabama,1,"Autauga County, AL",42963,2-3.9
1,1001,2000,Alabama,1,"Autauga County, AL",44021,4-5.9
2,1001,2001,Alabama,1,"Autauga County, AL",44889,4-5.9


In [24]:
#view data scale
Drug_Poison_County_df.shape

(56448, 7)

In [25]:
#display empty cells by column:
Drug_Poison_County_df.isnull().sum()

FIPS                                                            0
Year                                                            0
State                                                           0
FIPS State                                                      0
County                                                          0
Population                                                      0
Estimated Age-adjusted Death Rate, 16 Categories (in ranges)    0
dtype: int64

In [26]:
#save data in a new csv:
new_file_name_1 = os.path.join("csv_folder","clean_drug_pison_county.csv")
Drug_Poison_County_df.to_csv(new_file_name_1, index=False)

In [27]:
#4

In [28]:
#connect to the csv file:
Drug_Poison_State_csv = os.path.join("datadownload", "Drug_Poisoning_Mortality_by_State__United_States.csv")
#read csv file
drug_poison_state_df = pd.read_csv(Drug_Poison_State_csv, dtype=object)


In [29]:
#view dataframe
drug_poison_state_df.head(3)

,State,Year,Sex,Age Group,Race and Hispanic Origin,Deaths,Population,Crude Death Rate,Standard Error for Crude Rate,Lower Confidence Limit for Crude Rate,Upper Confidence Limit for Crude Rate,Age-adjusted Rate,Standard Error for Age-adjusted Rate,Lower Confidence Limit for Age-adjusted Rate,Upper Confidence Limit for Age-adjusted Rate,State Crude Rate in Range,US Crude Rate,US Age-adjusted Rate
0,Alabama,1999,Both Sexes,All Ages,All Races-All Origins,169,4430143,3.8148,0.29344,3.2396,4.3899,3.8521,0.29657,3.2708,4.4334,1.8–7.1,6.0382,6.057
1,Alabama,2000,Both Sexes,All Ages,All Races-All Origins,197,4447100,4.4299,0.31561,3.8112,5.0485,4.4857,0.31985,3.8588,5.1126,1.8–7.1,6.1882,6.1749
2,Alabama,2001,Both Sexes,All Ages,All Races-All Origins,216,4467634,4.8348,0.32896,4.19,5.4795,4.8915,0.33329,4.2382,5.5447,1.8–7.1,6.8057,6.7922


In [30]:
#view data scale
drug_poison_state_df.shape

(2862, 18)

In [31]:
#display empty cells by column:
drug_poison_state_df.isnull().sum()

State                                              0
Year                                               0
Sex                                                0
Age Group                                          0
Race and Hispanic Origin                           0
Deaths                                             0
Population                                         0
Crude Death Rate                                   0
Standard Error for Crude Rate                      0
Lower Confidence Limit for Crude Rate              0
Upper Confidence Limit for Crude Rate              0
Age-adjusted Rate                               1728
Standard Error for Age-adjusted Rate            1728
Lower Confidence Limit for Age-adjusted Rate    1728
Upper Confidence Limit for Age-adjusted Rate    1728
State Crude Rate in Range                       1728
US Crude Rate                                      0
US Age-adjusted Rate                               0
dtype: int64

In [32]:
#save data in a new csv:
new_file_name_1 = os.path.join("csv_folder","clean_drug_pison_state.csv")
drug_poison_state_df.to_csv(new_file_name_1, index=False)

In [33]:
#5

In [34]:
#connect to the Uniform_Crime_Reporting_System_Arrests by year csv file:
uCrime_Rep2015_csv = os.path.join("datadownload", "Uniform_Crime_Reporting_System_Arrests_2015.csv")
uCrime_Rep2014_csv = os.path.join("datadownload", "Uniform_Crime_Reporting_System_Arrests_2014.csv")
uCrime_Rep2013_csv = os.path.join("datadownload", "Uniform_Crime_Reporting_System_Arrests_2013.csv")
uCrime_Rep2012_csv = os.path.join("datadownload", "Uniform_Crime_Reporting_System_Arrests_in_2012.csv")
uCrime_Rep2010_csv = os.path.join("datadownload", "Uniform_Crime_Reporting_System_Arrests_in__2010.csv")

#read csv file
uCrime_Rep2015_df = pd.read_csv(uCrime_Rep2015_csv, dtype=object)
uCrime_Rep2014_df = pd.read_csv(uCrime_Rep2014_csv, dtype=object)
uCrime_Rep2013_df = pd.read_csv(uCrime_Rep2013_csv, dtype=object)
uCrime_Rep2012_df = pd.read_csv(uCrime_Rep2012_csv, dtype=object)
uCrime_Rep2010_df = pd.read_csv(uCrime_Rep2010_csv, dtype=object)

In [35]:
#view dataframe
uCrime_Rep2015_df.head(3)

,Arrestee,STAT INDEX,Andover,Ansonia,Ashford,Avon,Barkhamsted,Beacon Falls,Berlin,Bethany,...,New Haven County,New London County,Tolland County,Windham County,New York,Urban,NonUrban,CSP Total,NIBRS Data,Summary UCR Data
0,Murder <10,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Murder 10-12,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Murder 13-14,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
#view dataframe
uCrime_Rep2014_df.head(3)

,Description,stat_index,Connecticut,Andover,Ansonia,Ashford,Avon,Barkhamsted,Beacon Falls,Berlin,...,Wolcott,Woodbridge,Woodbury,Woodstock,Yale University,UConn - Storrs,Foxwoods Casino,Mohegan Sun Casino,MTA,C.C.S.U.
0,Murder <10,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Murder 10-12,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Murder 13-14,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
#view dataframe
uCrime_Rep2013_df.head(3)

,Description,Index,Connecticut,Andover,Ansonia PD,Ashford CSP,Avon PD,Barkhamsted CSP,Beacon Falls CSP,Berlin PD,...,New Haven,New London,Tolland,Windham,New York,Urban,NonUrban,CSP Total,NIBRS,Summary
0,Murder <10,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Murder 10-12,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Murder 13-14,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
#view dataframe
uCrime_Rep2012_df.head(3)

,Description,Index,Connecticut,Andover,Ansonia PD,Ashford CSP,Avon PD,Barkhamsted CSP,Beacon Falls CSP,Berlin PD,...,New Haven,New London,Tolland,Windham,New York,Urban,NonUrban,CSP Total,NIBRS,Summary
0,Murder <10,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Murder 10-12,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Murder 13-14,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
#view dataframe
uCrime_Rep2010_df.head(3)

,Description,Index,CT,Andover,Ansonia PD,Ashford CSP,Avon PD,Barkhamsted CSP,Beacon Falls CSP,Berlin PD,...,New Haven,New London,Tolland,Windham,New York,Urban,NonUrban,CSP Total,NIBRS,Summary
0,Murder <10,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Murder 10-12,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Murder 13-14,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
#view data scale Ucrime2015
uCrime_Rep2015_df.shape

(812, 203)

In [41]:
#display empty cells by column:
uCrime_Rep2015_df.isnull().sum()

Arrestee                  0
STAT INDEX                0
Andover                   0
Ansonia                   0
Ashford                   0
Avon                      0
Barkhamsted               0
Beacon Falls              0
Berlin                    0
Bethany                   0
Bethel                    0
Bethlehem                 0
Bloomfield                0
Bolton                    0
Bozrah                    0
Branford                  0
Bridgeport                0
Bridgewater               0
Bristol                   0
Brookfield                0
Brooklyn                  0
Burlington                0
Canaan                    0
Canterbury                0
Canton                    0
Chaplin                   0
Cheshire                  0
Chester                   0
Clinton                   0
Colchester                0
                         ..
Yale University           0
UConn - Storrs            0
Foxwoods Casino           0
Mohegan Sun Casino        0
MTA                 

In [42]:
#view data scale Ucrime2014
uCrime_Rep2014_df.shape

(812, 180)

In [43]:
#display empty cells by column:
uCrime_Rep2014_df.isnull().sum()

Description           0
stat_index            0
Connecticut           0
Andover               0
Ansonia               0
Ashford               0
Avon                  0
Barkhamsted           0
Beacon Falls          0
Berlin                0
Bethany               0
Bethel                0
Bethlehem             0
Bloomfield            0
Bolton                0
Bozrah                0
Branford              0
Bridgeport            0
Bridgewater           0
Bristol               0
Brookfield            0
Brooklyn              0
Burlington            0
Canaan                0
Canterbury            0
Canton                0
Chaplin               0
Cheshire              0
Chester               0
Clinton               0
                     ..
Voluntown             0
Wallingford           0
Warren                0
Washington            0
Waterbury             0
Waterford             0
Watertown             0
Westbrook             0
West Hartford         0
West Haven            0
Weston          

In [44]:
#view data scale Ucrime2013
uCrime_Rep2013_df.shape

(812, 202)

In [45]:
#display empty cells by column:
uCrime_Rep2013_df.isnull().sum()

Description               0
Index                     0
Connecticut               0
Andover                   0
Ansonia PD                0
Ashford CSP               0
Avon PD                   0
Barkhamsted CSP           0
Beacon Falls CSP          0
Berlin PD                 0
Bethany CSP               0
Bethel PD                 0
Bethlehem CSP             0
Bloomfield PD             0
Bolton CSP                0
Bozrah CSP                0
Branford PD               0
Bridgeport PD             0
Bridgewater CSP           0
Bristol PD                0
Brookfield PD             0
Brooklyn CSP              0
Burlington CSP            0
Canaan CSP                0
Canterbury CSP            0
Canton PD                 0
Chaplin CSP               0
Cheshire PD               0
Chester CSP               0
Clinton PD                0
                         ..
Woodbury CSP              0
Woodstock CSP             0
Yale Univ. PD             0
UCONN-Storrs PD           0
Foxwoods Casino CSP 

In [46]:
#view data scale Ucrime2012
uCrime_Rep2012_df.shape

(812, 202)

In [47]:
#display empty cells by column:
uCrime_Rep2012_df.isnull().sum()

Description               0
Index                     0
Connecticut               0
Andover                   0
Ansonia PD                0
Ashford CSP               0
Avon PD                   0
Barkhamsted CSP           0
Beacon Falls CSP          0
Berlin PD                 0
Bethany CSP               0
Bethel PD                 0
Bethlehem CSP             0
Bloomfield PD             0
Bolton CSP                0
Bozrah CSP                0
Branford PD               0
Bridgeport PD             0
Bridgewater CSP           0
Bristol PD                0
Brookfield PD             0
Brooklyn CSP              0
Burlington CSP            0
Canaan CSP                0
Canterbury CSP            0
Canton PD                 0
Chaplin CSP               0
Cheshire PD               0
Chester CSP               0
Clinton PD                0
                         ..
Woodbury CSP              0
Woodstock CSP             0
Yale Univ. PD             0
UCONN-Storrs PD           0
Foxwoods Casino CSP 

In [48]:
#view data scale Ucrime2010
uCrime_Rep2010_df.shape

(812, 202)

In [49]:
#display empty cells by column:
uCrime_Rep2010_df.isnull().sum()

Description               0
Index                     0
CT                        0
Andover                   0
Ansonia PD                0
Ashford CSP               0
Avon PD                   0
Barkhamsted CSP           0
Beacon Falls CSP          0
Berlin PD                 0
Bethany CSP               0
Bethel PD                 0
Bethlehem CSP             0
Bloomfield PD             0
Bolton CSP                0
Bozrah CSP                0
Branford PD               0
Bridgeport PD             0
Bridgewater CSP           0
Bristol PD                0
Brookfield PD             0
Brooklyn CSP              0
Burlington CSP            0
Canaan CSP                0
Canterbury CSP            0
Canton PD                 0
Chaplin CSP               0
Cheshire PD               0
Chester CSP               0
Clinton PD                0
                         ..
Woodbury CSP              0
Woodstock CSP             0
Yale Univ. PD             0
UCONN-Storrs PD           0
Foxwoods Casino CSP 

In [50]:
#

In [51]:
#connect to the csv file:
VSSR_csv = os.path.join("datadownload", "VSRR_Provisional_Drug_Overdose_Death_Counts.csv")
#read csv file
vssr_df = pd.read_csv(VSSR_csv, dtype=object)


In [52]:
#view dataframe
vssr_df.head(3)

,State,State Name,Year,Month,Period,Indicator,Data Value,Percent Complete,Percent Pending Investigation,Footnote
0,AK,Alaska,2015,January,12 month-ending,Number of Deaths,4034,100,0,NaN
1,AK,Alaska,2015,February,12 month-ending,Number of Deaths,4084,100,0,NaN
2,AK,Alaska,2015,March,12 month-ending,Number of Deaths,4101,100,0,NaN


In [53]:
#view data scale
vssr_df.shape

(8778, 10)

In [54]:
#display empty cells by column:
vssr_df.isnull().sum()


State                               0
State Name                          0
Year                                0
Month                               0
Period                              0
Indicator                           0
Data Value                          0
Percent Complete                    0
Percent Pending Investigation       0
Footnote                         6384
dtype: int64

In [55]:
#save data in a new csv:
new_file_name_1 = os.path.join("csv_folder","clean_vssr.csv")
vssr_df.to_csv(new_file_name_1, index=False)

In [56]:
#desired_state = False
#desired_state == vssr_df['CT']

#below is the working statment
#print(vssr_df[vssr_df['State']=='CT'])

In [57]:
#connect to the csv file:
ctEDU2014_2015_csv = os.path.join("datadownload", "State_Department_of_Education_Next_Generation_School_Accountability_data_2014-2015.csv")
#read csv file
ct_edu_2014_2015_df = pd.read_csv(ctEDU2014_2015_csv, dtype=object)


In [58]:
#view dataframe
ct_edu_2014_2015_df.head(3)

,District Name,District Code,School Name,School Code,School OrgType,School Low Grade,School High Grade,School Title I Type,Category,Total Points,...,Grad6YrHNNHNDiff,Threshold_Mean_ELA,Threshold_Mean_Math,Threshold_Mean_Sci,Threshold_Mean_Grad,Classification,State Category,# of Distinctions,School of Distinction - Highest Performing Overall,School of Distintion - Highest Performing High Needs
0,Andover School District,10011,District_0000000,0,District,District,NaN,NaN,DistrictTot,618.9,...,NaN,17.3,19.6,17.2,NaN,NaN,NaN,0,NaN,NaN
1,Andover School District,10011,Andover Elementary School_0010111,10111,Public Schools,PK,6,Targeted Assistance,SchoolTot,618.9,...,NaN,16.8,19.5,17.3,NaN,NaN,NaN,2,1,1
2,Ansonia School District,20011,District_0000000,0,District,District,NaN,NaN,DistrictTot,862.4,...,12.20%,17.3,19.6,17.2,15.2,NaN,NaN,0,NaN,NaN


In [59]:
#view data scale
ct_edu_2014_2015_df.shape

(1248, 88)

In [60]:
#display empty cells by column:
ct_edu_2014_2015_df.isnull().sum()


District Name                                              0
District Code                                              0
School Name                                                0
School Code                                                0
School OrgType                                             0
School Low Grade                                           1
School High Grade                                        205
School Title I Type                                      638
Category                                                   0
Total Points                                               0
Total Possible Points                                      0
Accountability Index                                       0
Achievement Gap Flag                                      90
Ind1 ELA_All_Rate                                        102
Ind1 ELA_All_Points                                      102
Ind1 ELA_All_PossiblePoints                               77
Ind1 ELA_HN_Rate        

In [61]:
#save data in a new csv:
new_file_name_1 = os.path.join("csv_folder","clean_ct_edu_2014_2015.csv")
ct_edu_2014_2015_df.to_csv(new_file_name_1, index=False)

In [62]:
#connect to the csv file:
CT_Unemployment_xlsx = os.path.join("datadownload", "CT Unemployment Rates.xlsx")

#read csv file
ct_unemployment_df = pd.read_excel(CT_Unemployment_xlsx, dtype=object)

In [63]:
#view dataframe
ct_unemployment_df.head(4)

,Month,1984,1985,1986,1987,1988,1989,1990,1991,1992,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Jan,4.9,4.9,3.9,3.8,2.9,3.1,4.8,5.5,7.5,...,6.9,9,9.2,8.2,8.2,7.2,6.1,5.5,4.9,4.5
1,Feb,4.8,4.9,3.9,3.7,2.9,3.1,4.9,5.7,7.5,...,7.1,9,9.2,8.2,8.1,7,6,5.5,4.9,4.6
2,Mar,4.7,5,3.9,3.6,2.8,3.2,5,5.8,7.5,...,7.3,9.1,9.2,8.3,8.1,6.9,5.9,5.4,4.9,4.5
3,Apr,4.6,5,3.9,3.5,2.8,3.3,5,6,7.5,...,7.5,9.1,9.1,8.3,8,6.8,5.8,5.3,4.8,NaN


In [64]:
#view data scale
ct_unemployment_df.shape

(12, 36)

In [65]:
#display empty cells by column:
ct_unemployment_df.isnull().sum()

Month    0
1984     0
1985     0
1986     0
1987     0
1988     0
1989     0
1990     0
1991     0
1992     0
1993     0
1994     0
1995     0
1996     0
1997     0
1998     0
1999     0
2000     0
2001     0
2002     0
2003     0
2004     0
2005     0
2006     0
2007     0
2008     0
2009     0
2010     0
2011     0
2012     0
2013     0
2014     0
2015     0
2016     0
2017     0
2018     9
dtype: int64

In [66]:
#save data in a new csv:
new_file_name_1 = os.path.join("csv_folder","clean_ct_unemployment.csv")
ct_unemployment_df.to_csv(new_file_name_1, index=False)

In [67]:
#connect to the csv file:
USA_Unemployment_xlsx = os.path.join("datadownload", "USA Unemployment Rates.xlsx")

#read csv file
usa_unemployment_df = pd.read_excel(USA_Unemployment_xlsx, dtype=object)

In [68]:
#view dataframe
usa_unemployment_df.head(4)

,Month,1984,1985,1986,1987,1988,1989,1990,1991,1992,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Jan,8,7.3,6.7,6.6,5.7,5.4,5.4,6.4,7.3,...,7.8,9.8,9.1,8.3,8,6.6,5.7,4.9,4.8,4.1
1,Feb,7.8,7.2,7.2,6.6,5.7,5.2,5.3,6.6,7.4,...,8.3,9.8,9,8.3,7.7,6.7,5.5,4.9,4.7,4.1
2,Mar,7.8,7.2,7.2,6.6,5.7,5,5.2,6.8,7.4,...,8.7,9.9,9,8.2,7.5,6.7,5.5,5,4.5,4.1
3,Apr,7.7,7.3,7.1,6.3,5.4,5.2,5.4,6.7,7.4,...,9,9.9,9.1,8.2,7.6,6.3,5.4,5,4.4,NaN


In [69]:
#view data scale
usa_unemployment_df.shape

(12, 36)

In [70]:
#display empty cells by column:
usa_unemployment_df.isnull().sum()

Month    0
1984     0
1985     0
1986     0
1987     0
1988     0
1989     0
1990     0
1991     0
1992     0
1993     0
1994     0
1995     0
1996     0
1997     0
1998     0
1999     0
2000     0
2001     0
2002     0
2003     0
2004     0
2005     0
2006     0
2007     0
2008     0
2009     0
2010     0
2011     0
2012     0
2013     0
2014     0
2015     0
2016     0
2017     0
2018     9
dtype: int64

In [71]:
#save data in a new csv:
new_file_name_1 = os.path.join("csv_folder","clean_usa_unemployment.csv")
usa_unemployment_df.to_csv(new_file_name_1, index=False)

In [72]:
#connect to the csv file:
HomicideRates_2009_2013_xlsx = os.path.join("datadownload", "expanded_homicide_data_table_8_murder_victims_by_weapon_2009-2013.xls")

#read csv file
homicideRates_2009_2013_df = pd.read_excel(HomicideRates_2009_2013_xlsx, dtype=object)

In [73]:
#view dataframe
homicideRates_2009_2013_df.head(4)

,Expanded Homicide Data Table 8,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Murder Victims,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"by Weapon, 2009–2013",NaN,NaN,NaN,NaN,NaN,NaN,
2,Weapons,2009,2010,2011,2012,2013,NaN,NaN
3,Total,13752,13164,12795,12888,12253,,


In [74]:
#view data scale
homicideRates_2009_2013_df.shape

(25, 8)

In [75]:
#display empty cells by column:
homicideRates_2009_2013_df.isnull().sum()

Expanded Homicide Data Table 8     2
Unnamed: 1                         6
Unnamed: 2                         6
Unnamed: 3                         6
Unnamed: 4                         6
Unnamed: 5                         6
Unnamed: 6                        22
Unnamed: 7                        22
dtype: int64

In [76]:
#save data in a new csv:
new_file_name_1 = os.path.join("csv_folder","homicideRates.csv")
homicideRates_2009_2013_df.to_csv(new_file_name_1, index=False)

In [77]:
#